In [15]:
from modules.utils.s3 import get_from_s3, put_to_s3, list_s3
from constants import BUCKET, DATA_CLEAN_PREFIX, DATA_RAW_PREFIX, DATA_PROCESSED_PREFIX
import json
import re

In [40]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet


nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word: str):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    return "v" if tag in ["j", "r", "v"] else "n"

In [ ]:
lemmatizer.lemmatize("subproducts", wordnet.NOUN)

In [ ]:
get_wordnet_pos("continued")

In [ ]:
nltk.pos_tag(["new"])

In [248]:
bytes(pd.DataFrame([[1,2,3],[4,5,6]]).to_csv().encode())

b',0,1,2\n0,1,2,3\n1,4,5,6\n'

In [257]:
pd.DataFrame(np.array([[1,2,3], [1,2,3]])).to_csv()

',0,1,2\n0,1,2,3\n1,1,2,3\n'

In [254]:
np.array2string(np.array([[1,2,3],[4,5,6]]), precision=2, separator=', ')

'[[1, 2, 3],\n [4, 5, 6]]'

In [232]:
bytes(str(np.array([[1,2,4],[2,4,5]])), encoding="UTF-8")

b'[[1 2 4]\n [2 4 5]]'

In [10]:
response = get_from_s3(BUCKET, f"{DATA_CLEAN_PREFIX}dataset.json")
all_entries = json.loads(response)

In [267]:
index = 97

In [268]:
text = all_entries[index]["body"]
text

'Shr profit three cts vs loss three cts Net profit 363,000 vs loss 197,000 Revs 3,761,000 vs 2,666,000 Year Shr profit one cent vs loss 37 cts Net profit 129,000 vs loss 1,715,000 Revs 11.4 mln vs 10.9 mln Avg shrs 10,694,081 vs 4,673,253 Reuter '

In [195]:
text = all_entries[index]["body"]
text = text.lower()
text = re.sub(r"[u][\.][s][\.]", "usa", text)
text = re.sub(r"\.", "", text)  # Remove periods, this will squash abbreviations ("l.p." -> "lp")
text = re.sub(r"[0-9]", "", text)  # Remove numbers
text = re.sub(r"[\W_]", " ", text)  # Turn any non-alphanumber character into space
text = re.sub(r"\s+", r" ", text)  # Reduce continuous spaces
text

'showers continued throughout the week in the bahia cocoa zone alleviating the drought since early january and improving prospects for the coming temporao although normal humidity levels have not been restored comissaria smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february were bags of kilos making a cumulative total for the season of mln against at the same stage last year again it seems that cocoa delivered earlier on consignment was included in the arrivals figures comissaria smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end with total bahia crop estimates around mln bags and sales standing at almost mln there are a few hundred thousand bags still in the hands of farmers middlemen exporters and processors there are doubts as to how much of this cocoa would be fit for export as shippers are now experiencing dificulties in obtaining 

In [196]:
# # Lower text
# text = text.lower()
# # Remove white spaces
# text = str(text).replace("\n", " ")
# text = re.sub(r'[^\w\s]',' ',text)
# # Remove non-alphabetical characters 
# text = re.sub('[0-9]',"",text)
# text = re.sub(" +", " ", text)
# text = re.sub("([^\x00-\x7F])+"," ",text)
# Split into words
text = text.split()
# Remove stopwords
text = [word for word in text if word not in ["reuter"] + stopwords.words("english")]

In [197]:
" ".join(text)

'showers continued throughout week bahia cocoa zone alleviating drought since early january improving prospects coming temporao although normal humidity levels restored comissaria smith said weekly review dry period means temporao late year arrivals week ended february bags kilos making cumulative total season mln stage last year seems cocoa delivered earlier consignment included arrivals figures comissaria smith said still doubt much old crop cocoa still available harvesting practically come end total bahia crop estimates around mln bags sales standing almost mln hundred thousand bags still hands farmers middlemen exporters processors doubts much cocoa would fit export shippers experiencing dificulties obtaining bahia superior certificates view lower quality recent weeks farmers sold good part cocoa held consignment comissaria smith said spot bean prices rose cruzados per arroba kilos bean shippers reluctant offer nearby shipment limited sales booked march shipment dlrs per tonne port

In [269]:
# {word: nltk.pos_tag([word])[0][1][0] for word in text}
{word: get_wordnet_pos(word) for word in text}

{'S': 'n',
 'h': 'n',
 'r': 'n',
 ' ': 'n',
 'p': 'n',
 'o': 'n',
 'f': 'n',
 'i': 'n',
 't': 'n',
 'e': 'n',
 'c': 'n',
 's': 'n',
 'v': 'n',
 'l': 'n',
 'N': 'n',
 '3': 'n',
 '6': 'n',
 ',': 'n',
 '0': 'n',
 '1': 'n',
 '9': 'n',
 '7': 'n',
 'R': 'n',
 '2': 'n',
 'Y': 'n',
 'a': 'n',
 'n': 'n',
 '5': 'n',
 '.': 'n',
 '4': 'n',
 'm': 'n',
 'A': 'n',
 'g': 'n',
 '8': 'n',
 'u': 'n'}

In [199]:
# Lemmatize
text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in text]
text

['shower',
 'continue',
 'throughout',
 'week',
 'bahia',
 'cocoa',
 'zone',
 'alleviate',
 'drought',
 'since',
 'early',
 'january',
 'improve',
 'prospect',
 'come',
 'temporao',
 'although',
 'normal',
 'humidity',
 'level',
 'restore',
 'comissaria',
 'smith',
 'say',
 'weekly',
 'review',
 'dry',
 'period',
 'mean',
 'temporao',
 'late',
 'year',
 'arrival',
 'week',
 'end',
 'february',
 'bag',
 'kilo',
 'make',
 'cumulative',
 'total',
 'season',
 'mln',
 'stage',
 'last',
 'year',
 'seem',
 'cocoa',
 'deliver',
 'earlier',
 'consignment',
 'include',
 'arrival',
 'figure',
 'comissaria',
 'smith',
 'say',
 'still',
 'doubt',
 'much',
 'old',
 'crop',
 'cocoa',
 'still',
 'available',
 'harvest',
 'practically',
 'come',
 'end',
 'total',
 'bahia',
 'crop',
 'estimate',
 'around',
 'mln',
 'bag',
 'sale',
 'stand',
 'almost',
 'mln',
 'hundred',
 'thousand',
 'bag',
 'still',
 'hand',
 'farmer',
 'middleman',
 'exporter',
 'processor',
 'doubt',
 'much',
 'cocoa',
 'would',
 'f